In [2]:
# A first-level heading
## A second-level heading
### A third-level heading


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
!pip install --upgrade xlrd #colab bug verson xlrd
!pip install geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install contextily
!pip install pygeos
!pip install topojson
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
#!pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
#!pip install git+https://github.com/inseefrlab/cartiflette@80b8a5a28371feb6df31d55bcc2617948a5f9b1a

In [ ]:
#Idée générale : on va reprendre le graphe stat desc pour identifier la source de pollution principale dans le Finistère

In [ ]:
eff = pd.read_csv("effectifs.csv", sep = ";", encoding = 'utf-8', engine = 'python')

In [ ]:
data_pollution = pd.read_csv("base_pollution.csv", sep = ",", encoding = 'utf-8', engine = 'python')

In [ ]:
pollution_group_dep = pd.read_csv("pollution_group_dep.csv", sep = ",", encoding = 'utf-8', engine = 'python')

In [ ]:
pollution_group_reg = pd.read_csv("pollution_group_reg.csv", sep = ",", encoding = 'utf-8', engine = 'python')

In [ ]:
x = input("Entrer un numéro de département (mettre un 0 devant si c'est un chiffre) : ")

In [ ]:
# Sélectionnez les données pour le département x
pollution_dep_91 = pollution_group_dep[pollution_group_dep['DEP'] == x]

# Sélectionnez les colonnes correspondant aux sources de pollution
sources_pollution = pollution_dep_91.columns[1:10]

# Créez un diagramme en barres
plt.bar(sources_pollution, pollution_dep_91.iloc[0, 1:10])
plt.xlabel('Sources de pollution')
plt.ylabel('Niveau de pollution')
plt.title('Niveau de pollution par source pour le département {}'.format(x))
plt.xticks(rotation=45, ha='right')

plt.show()


#La source de pollution principale dans le Finistère est l'agriculture 

In [ ]:
#Pa rapport aux autres histogramme ici on va prendre pathoniv2 pour les dummies 
base_grande = pd.read_csv("effectifs.csv", sep = ";", encoding = 'utf-8', engine = 'python')
base_grande.shape
base_grande = base_grande.dropna()
base_grande = pd.get_dummies(base_grande, prefix = 'indicatrice_', columns=['patho_niv2'], dtype='int')


liste_var = base_grande.columns[17:]
liste_bis = []
for i in liste_var : 
    liste_bis.append(i)
indicatrice_patho_niv1 = liste_bis

liste_auxil = base_grande['cla_age_5'].unique()
inter_excl = ('95et+', 'tsage', np.nan)

inter_def = []
for i in liste_auxil :
    if i in inter_excl : 
        inter_def = inter_def
    else : 
        inter_def.append(i)
classe_age = []
for i in base_grande['cla_age_5'] : 
    if i in inter_def :
        temp = i.split('-')
        temp[0] = int(temp[0])
        temp[1] = int(temp[1])
        age = (temp[1] + temp[0])/2
    elif i == 'tsage' : 
        age = np.nan
    elif i == '95et+' : 
        age = 100
    else : 
        age = np.nan
    classe_age.append(age)
base_grande.loc[:,'classe_age'] = classe_age

In [ ]:
#On supprime les espaces dans le noms des colonnes 
base_grande.columns = base_grande.columns.str.replace(' ', '_')

In [ ]:
#On recréer un base similaire auxx base de statdesc
model_prostate = base_grande[['dept', 'classe_age','indicatrice__Cancer_de_la_prostate','sexe','region']]
model_prostate

In [ ]:
compte_dept_29 = 0

for index, row in model_prostate.iterrows():
    if row['dept'] == '29':
        compte_dept_29 += 1

print("Nombre d'individus avec dept=='29' :", compte_dept_29)

#Ce code nous donne le nombre d'individus en provenance du Finistère à savoir 18467

In [ ]:
compte_prostate = 0

# Supposons que model_prostate soit votre DataFrame
# Remplacez-le par le nom de votre DataFrame
for index, row in model_prostate.iterrows():
    if row['dept'] == '29' and row['indicatrice__Cancer_de_la_prostate'] == 1:
        compte_prostate += 1

print("Nombre d'individus avec dept=='29' et indicatrice__Cancer_de_la_prostate==1 :", compte_prostate)

#Ce code nous donne le nombre d'individus en provenance du Finistère et qui sont atteints d'un cancer de la prostate à savoir 320

#Oa donc 320/18467*100 soit environ 0,17% de cancer de la prostate parmi les observés résidants dans le Finistère 


In [ ]:
#On prend la liste des départements observés afin de généralisé notre démarche
valeurs_uniques = model_prostate['dept'].unique()
print(valeurs_uniques)

In [ ]:
import pandas as pd

# Supposons que 'model_prostate' est votre DataFrame
# et 'dept' est la colonne représentant les départements

# Grouper par département et compter le nombre total d'individus et le nombre de cas de cancer de la prostate
resultats = model_prostate.groupby('dept')['indicatrice__Cancer_de_la_prostate'].agg(['count', 'sum'])

# Renommer les colonnes
resultats.columns = ['pop_i', 'pop_i2']

# Remplacer les valeurs manquantes par 0
resultats['pop_i2'].fillna(0, inplace=True)

# Initialiser la variable pour la moyenne
moyenne_totale = 0

# Calculer le pourcentage de cancer de la prostate par département et afficher les résultats
for index, row in resultats.iterrows():
    if row['pop_i'] != 0:
        pourcentage_cancer = (row['pop_i2'] / row['pop_i']) * 100
        moyenne_totale += pourcentage_cancer
        print(f"Il y a {pourcentage_cancer:.2f}% de cancer de la prostate dans le département {index}.")
    else:
        print(f"Aucun individu dans le département {index} dans votre DataFrame.")

# Calculer la moyenne totale
if len(resultats) > 0:
    moyenne_totale /= len(resultats)
    print(f"\nLa moyenne totale des pourcentages de cancer de la prostate est {moyenne_totale:.2f}%.")
else:
    print("\nAucun résultat à moyenner, car tous les départements ont une population de 0.")


In [ ]:
#On observe donc :
#Des résultats inférieurs à la moyenne dans le Finistère
#Des résultats très élevés dans les outre mer
#Des résultats au dessus de 2% pour le Gers(32), la Creuse(23), et l'Ariège(09) le Lot(46) et la Lozère(48) pour la France métropolitaine

In [ ]:
#En observant l'histogramme des souces de pollution pour ces différents départements, on remarque que :
#L'agriculture est la source principale de pollution
